# Exploratory analysis of the HHL linear solver

## Implementation in `qiskit`

`qiskit` has an implementation of the hhl algorithm.
It can be imported using:

In [1]:
from qiskit.algorithms.linear_solvers.hhl import HHL

## Minimal example

We'll begin by importing a couple of dependencies.

In [2]:
import numpy as np
from qiskit.algorithms.linear_solvers.numpy_linear_solver import NumPyLinearSolver
from qiskit.algorithms.linear_solvers.matrices.tridiagonal_toeplitz import TridiagonalToeplitz
from qiskit.quantum_info import Statevector

## Problem posing

In this example, based on the [`qiskit` tutorials](https://qiskit.org/textbook/ch-applications/hhl_tutorial.html), we'll solve a simple linear system of the form:

$$
A \vec x = \vec b
$$

where:

$$
A =
\begin{bmatrix}
a & b \\
b & a
\end{bmatrix}
:
a = 1,
b = -\frac{1}{3}
$$

and

$$
\vec b = 
\begin{bmatrix}
1 \\
0
\end{bmatrix}
$$

In [3]:
a = 1
b = -1/3
matrix = np.array([[a, b], [b, a]])
vector = np.array([1, 0])

### Classical solution

Keep in mind that during the encoding the vector will be normalized.
This means that the equivalent classical problem would be that of:

$$
A\vec x = \frac{\vec b}{\| \vec b\|} \equiv \hat b
$$

In [4]:
nvector = vector / np.linalg.norm(vector)
classical_solution = NumPyLinearSolver().solve(matrix, nvector)
print(classical_solution)

{   'circuit_results': None,
    'euclidean_norm': 1.1858541225631423,
    'observable': None,
    'state': array([1.125, 0.375])}


The solution is given by:

In [5]:
exact_solution = classical_solution.state
print(exact_solution)

[1.125 0.375]


We can check that the classical solution $\vec s$ is correct by evaluating ensuring that:

$$
A \vec s - \hat b = \vec 0
$$

In [6]:
assert np.max(np.matmul(matrix, classical_solution.state) - nvector) == 0.0

## HHL solution

The `HHL` method has a very similar interface to that of `NumPyLinearSolver`, but applied naively, it will return the circuit instead of the solution:

### Naive approach

In [7]:
naive_hhl_solution = HHL().solve(matrix, vector)
print(naive_hhl_solution.state)

      ┌───────────┐┌──────┐        ┌─────────┐
q0_0: ┤ circuit-7 ├┤2     ├────────┤2        ├
      └───────────┘│      │┌──────┐│         │
q1_0: ─────────────┤0 QPE ├┤1     ├┤0 QPE_dg ├
                   │      ││      ││         │
q1_1: ─────────────┤1     ├┤0 1/x ├┤1        ├
                   └──────┘│      │└─────────┘
q2_0: ─────────────────────┤2     ├───────────
                           └──────┘           


In order to peek into the solutions, we can proceed as follows:

In [8]:
''' Auxiliary function to extract the full solution as an array

Based on https://qiskit.org/textbook/ch-applications/hhl_tutorial.html
'''
def get_solution_as_array(qc, isreal=True):
    # Extract state as Statevector
    statevec = Statevector(qc.state).data
    
    # Extract the relevant information TODO: why is it so obscure?
    solvec = np.array([statevec[8], statevec[9]])

    # If we expect real values ...
    if isreal: # ... get rid of the imaginary parts
        solvec = np.real(solvec)

    # Fix possible rescalings made by the circuit
    solvecrenorm = qc.euclidean_norm * solvec / np.linalg.norm(solvec)

    return solvecrenorm


In [9]:
print(get_solution_as_array(naive_hhl_solution))

[1.125 0.375]


### Using `TridiagonalToeplitz`

It is advisable to encode the problem using a `TridiagonalToeplitz` matrix instead.
The equivalent matrix for our problem will be:

$$
A =
\begin{bmatrix}
a & b & 0 & 0 \\
b & a & b & 0 \\
0 & b & a & b \\
0 & 0 & b & a
\end{bmatrix}
$$

Keep in mind that for problems larger than $2 \times 2$ the results of using `TridiagonalToeplitz` would be approximate.

In [10]:
tridi_matrix = TridiagonalToeplitz(num_state_qubits=1, main_diag=a, off_diag=b)
tridi_solution = HHL().solve(tridi_matrix, vector)
print(get_solution_as_array(tridi_solution))

[1.125 0.375]


## Observables

In real quantum computers, we cannot know the full state vector; we have to compute observables instead.

So far, the only implemented operators for `linear_solvers` are the euclidean norm, `AbsoluteAverage` and `MatrixFunctional`.
As usual, we'll need to import them to begin with.

In [11]:
from qiskit.algorithms.linear_solvers.observables import AbsoluteAverage, MatrixFunctional

The euclidean norm is returned automatically, and doesn't require importing any operator.
It returns:

$$
{\tt norm} (\vec x) = \sqrt{\sum_{i=1}^N x_i^2 }
$$

In [12]:
print("Euclidean norm with HHL :", naive_hhl_solution.euclidean_norm)
print("Euclidean norm (exact)  :", np.linalg.norm(exact_solution))

Euclidean norm with HHL : 1.1858541225631414
Euclidean norm (exact)  : 1.1858541225631423


The `AbsoluteAverage` operator returns:

$$
{\tt AbsoluteAverage}(\vec x) = \| \frac{1}{N} \sum_{i=1}^N x_i \|
$$

In [13]:
observable = AbsoluteAverage()
average_hhl_solution = HHL().solve(matrix, vector, observable)

We can compare the result with the exact value:

In [14]:
print("Average with HHL :", average_hhl_solution.observable)
print("Average (exact)  :", np.mean(exact_solution))

Average with HHL : 0.7499999999999993
Average (exact)  : 0.75


The `MatrixFunctional` operator contains a tridiagonal matrix $B$, and returns:

$$
{\tt MatrixFunctional} (\vec x; a, b) = \vec x^T B(a, b) \vec x 
$$

where:

$$
B(a,b) = 
\begin{bmatrix}
a & b & 0 & 0 & \dots \\
b & a & b & 0 & \dots \\
0 & b & a & b & \dots \\
0 & 0 & b & a & \dots \\
\vdots & \vdots & \vdots & \vdots & \ddots

\end{bmatrix}
$$

In [15]:
observable = MatrixFunctional(a, b)
mf_hhl_solution = HHL().solve(matrix, vector, observable)

In [16]:
print("Matrix functional with HHL :", mf_hhl_solution.observable)
print("Matrix functional (exact)  :", np.matmul(np.transpose(exact_solution), np.matmul(matrix, exact_solution)))

Matrix functional with HHL : 1.124999999999999
Matrix functional (exact)  : 1.125
